In [ ]:
#importing libraries
import sys
import time
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

In [ ]:
#function for scrapping pharmeasy data
def scrape_pharmeasy_data(url):
    #write the scraping code for pharmeasy here
    try:
        page = requests.get(url)
    except Exception as e:
        error_type, error_info, error_obj = sys.exc_info()
        print('ERROR FOR LINK', url, '\n')
        print(error_type, ' Line: ', error_info.tb_lineno)

    time.sleep(2)

    soup = BeautifulSoup(page.text, 'html.parser')
    
    try:
        name = soup.find('div', {'class': 'MedicineOverviewSection_nameContainer__du_iv'}).text
    except AttributeError:
        name = ' '
    #mrp = soup.find('span', {'class': 'PriceInfo_striked__Hk2U_'}).text

    try:
        mrp_element = soup.find('span', {'class': 'PriceInfo_striked__Hk2U_'})
        mrp = mrp_element.text if mrp_element else 'N/A'
    except AttributeError:
        mrp = ' '

    try:
        marketer = soup.find('div', {'class': 'MedicineOverviewSection_brandName__rJFzE'}).text
    except AttributeError:
        marketer = ' '

        
    table = soup.find('table', {'class': 'DescriptionTable_seoTable__wKp77'})
    if table:
        rows = table.find_all('tr')
        for row in rows:
            if row.find('td', {'class': 'DescriptionTable_field__l5jJ3'}).text == 'Offer Price':
                discounted_price = row.find('td', {'class': 'DescriptionTable_value__0GUMC'}).text
            elif row.find('td', {'class': 'DescriptionTable_field__l5jJ3'}).text == 'Contains':
                salts = row.find('td', {'class': 'DescriptionTable_value__0GUMC'}).text
                break
        else:
            salts = None
    else:
        discounted_price = None

    url = url

    return name, mrp, discounted_price, marketer, salts, url

In [ ]:
#function for scrapping netmeds data
def scrape_netmeds_data(url):
    #write the scraping code for netmeds here
    try:
        page = requests.get(url)
    except Exception as e:
        error_type, error_info, error_obj = sys.exc_info()
        print('ERROR FOR LINK', url, '\n')
        print(error_type, ' Line: ', error_info.tb_lineno)

    time.sleep(2)
    soup = BeautifulSoup(page.text, 'html.parser')
        
    name = soup.find('h1', {'class': 'black-txt'}).text
    mrp = soup.find('span', {'class': 'final-price'}).text
    discounted_price = soup.find('span', {'class': 'barBestPrice'}).text
    
    marketer_div = soup.find('div', string='Name of Manufacturer/Marketer')
    if marketer_div:
        marketer = marketer_div.find_next('div', {'class': 'manufacturer__name_value'}).text
    else:
        marketer = 'Not found'
    
    # Change the salts
    salts_div = soup.find('div', string='Ingredient')
    if salts_div:
        salts = salts_div.find_next('div', {'class': 'manufacturer__name_value'}).text
    else:
        salts = 'Not found'

    url = url

    return name, mrp, discounted_price, marketer, salts, url 

In [ ]:
url = 'https://pharmeasy.in/online-medicine-order/browse?alphabet=b&page=0' 
response = requests.get(url)

if response.status_code == 200:
    html_content = response.text
else:
    print("Failed to fetch the webpage")

In [ ]:
soup = BeautifulSoup(html_content, 'html.parser')

# Find all anchor tags with class 'BrowseList_medicine__cQZkc'
medicine_links = soup.find_all('a', class_='BrowseList_medicine__cQZkc')

# List to store extracted URLs
urls = []

# Extract URLs
for link in medicine_links:
    href = link.get('href')
    # Check if href starts with '/online-medicine-order/'
    if href.startswith('/online-medicine-order/'):
        # Construct the full URL and append it to the list
        full_url = f'https://pharmeasy.in{href}'
        urls.append(full_url)

# Function to scrape data from a given URL (You need to implement this function)
def scrape_pharmeasy_data(url):
    # Your scraping logic here
    pass

# List to store scraped data
data = []

# Iterate through the extracted URLs and scrape data
for url in urls:
    pharmeasy_data = scrape_pharmeasy_data(url)
    data.append(pharmeasy_data)

# At this point, 'data' should contain the scraped data from all the URLs
# You can then process this data as needed.


In [ ]:
urls

In [ ]:
data = []

In [ ]:
scrape_pharmeasy_data(urls[0])

In [ ]:
for url in urls:
    output = scrape_pharmeasy_data(url)
    data.append(output)

In [ ]:
# Flatten the nested list of dictionaries
flat_data = [item for sublist in data for item in sublist if sublist]

# Write data to CSV file
csv_filename = 'medicines.csv'
if flat_data:
    keys = flat_data[0].keys()  # Extract keys for header
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(flat_data)
        print(f"Data written to {csv_filename}")
else:
    print("No data to write to CSV")